In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

In [2]:
import pandas as pd
import os
import numpy as np
from scipy.signal import savgol_filter
import plotly.graph_objects as go
from scipy.integrate import simps

In [3]:
# Función para calculo de aceleracion. h ~ datos por segundo
def calcular_acel(df,h): # en m/s2
    vel = df["v_km_h"]
    v = np.array(vel)/3.6  # Pasa de km/h a m/s
    n = len(v)
    a = np.zeros(n)
    for i in range(n):
        if i == 0:
            a[i]=(-v[i+2] + 4*v[i+1] - 3*v[i])/(2*h)
            #print("i=0")
        elif i == 1:
            a[i]=(v[i+1] - v[i-1])/(2*h)
            #print("i=1")
        elif i == n-2:
            a[i]=(v[i+1] - v[i-1])/(2*h)
            #print("i=n-2")
        elif i == n-1:
            a[i]=(3*v[i] - 4*v[i-1] + v[i-2])/(2*h)
            #print("i=n-1")
        else:
            a[i] = (-v[i+2] + 8*v[i+1] - 8*v[i-1] + v[i-2])/(12*h)
    return(a)

import pandas as pd

def calcular_parametros_acel(df):
    Data_time = 1 # Un dato por segundo
    df = df.reset_index(drop=True) # Reindexar el DF
    df["v_m_s"] = df["v_km_h"]/3.6
    df["t_s"] = range(len(df)) # Crear nueva columna de tiempo con una frecuencia de 1hz
    df["acel_m_s2"] = calcular_acel(df,Data_time) 
    df['ABS_Aceleracion'] = abs(df['acel_m_s2'])
    df['Acel_cumul'] = df['ABS_Aceleracion'].cumsum()
    df['Derivada_Aceleracion'] = df['acel_m_s2'].diff()
    df['Derivada_Aceleracion'].fillna(0, inplace=True)
    df['ABS_Derivada_Aceleracion'] = abs(df['Derivada_Aceleracion'])
    df['Der_Acel_cumul'] = df['ABS_Derivada_Aceleracion'].cumsum()
    return df

def parametros_seccion(df1): 
    umbral = 0.1 #con valores de aceleración mayor a 0.1 m/s2. para ia, RPA y crucero
    df1["t_s"] = range(len(df1)) # Crear nueva columna de tiempo con una frecuencia de 1hz
    # Calcular la distancia recorrida
    df1["v_m_s"] = df1["v_km_h"]/3.6
    distance_traveled = (simps(df1["v_m_s"].values, df1.t_s.values))/1000  # En km
    # Calcular el tiempo total del recorrido
    total_time = (df1["t_s"].iloc[-1] - df1["t_s"].iloc[0])
    # Calcular la velocidad media
    speed_mean = (simps(df1["v_km_h"], df1["t_s"] * 1)) / (len(df1["t_s"]))
    # Calcular la velocidad máxima
    speed_max = df1["v_km_h"].max()
    # Calcular la velocidad máxima
    speed_mean_sup = (df1["v_km_h"].max()+speed_mean)/2
    # Calcular la aceleración media y máxima tanto negativa como positiva
    acel_pos_mean = df1[df1.acel_m_s2 > 0].acel_m_s2.mean()
    acel_pos_max = df1[df1.acel_m_s2 > 0].acel_m_s2.max()
    acel_neg_mean = df1[df1.acel_m_s2 <= 0].acel_m_s2.mean()
    acel_neg_max = df1[df1.acel_m_s2 <= 0].acel_m_s2.min()
    # Calcular tiempo en ralentí, tiempo en movimiento, tiempo en crucero y tiempo en aceleracion
    time_ralenti = (df1[(df1['v_km_h'] == 0)].shape[0]) * (100 / total_time)
    time_mov = (df1[(df1['v_km_h'] > 0)].shape[0]) * (100 / total_time)
    t_crucero = (df1[(df1['ABS_Aceleracion'] <= 0.1) & (df1['v_km_h'] > 0)].shape[0]) * (100 / total_time)
    t_acel = time_mov - t_crucero
    # Funcion que cuenta el numero de veces con valores de aceleración mayor a 0.1 m/s2.   
  
    df_filtered = df1[df1['acel_m_s2'] > umbral]
    n_datos_pos = len(df_filtered)
    df_filtered = df1[df1['acel_m_s2'] < -umbral]
    n_datos_neg = len(df_filtered)
    n_ia_t = n_datos_pos + n_datos_neg
    ## CALCULAR RPA
    # Calcular el producto de la velocidad por la aceleración [m3/s3 o W/kg]
    df1['vxa'] = df1['v_m_s']*df1["acel_m_s2"]
    # Calcular la distnacia recorrida del intervalo en metros
    distance_traveled =  (simps(df1["v_km_h"].divide(3.6).values,df1.t_s.values)) 
    # Calculo de velocidad media en cada intervalo         
    m_v_urban = df1['v_km_h'].mean()  
    # Extraer solo el dataframe con acel mayor a 0.1
    df_sort_Urban = df1[df1["acel_m_s2"]>0.1]
    # Ordenar de manera creciente de la vxa
    df_sort_vxa_urb = df_sort_Urban.sort_values(by='vxa')
    #Percentil para cada intervalo
    df_sort_vxa_urb['Percentil'] = range(1, len(df_sort_vxa_urb) + 1)
    df_sort_vxa_urb['Percentil']  = df_sort_vxa_urb['Percentil']/len(df_sort_vxa_urb)
    # interpolacion lineal para calcular el valor vxa para 0.95 en cada intervalo
    vxa_pos_urbano = df_sort_vxa_urb['vxa'].quantile(0.95, interpolation='linear')
    #Aceleració positiva relativa RPA
    RPA = (sum(df_sort_vxa_urb["vxa"]))/(distance_traveled)
    # Acumulado del valor absoluto de la derivada de la aceleracion
    ABS_Der_Acel = df1["Der_Acel_cumul"].max() #Acumulado del valor absoluto de la derivada de la aceleracion
    # Acumulado del valor absoluto de la derivada de la aceleracion por cada km recorrido
    ABS_Der_Acel_Per_distance = ABS_Der_Acel/(distance_traveled) #ABS_Der_Acel en cada km recorrido
    # Acumulado del valor absoluto de la aceleracion 
    ABS_Acel = df1['Acel_cumul'].max() #Acumulado del valor absoluto de la aceleracion
    # Acumulado del valor absoluto de la aceleracion por cada km recorrido
    ABS_Acel_Per_distance = ABS_Acel/(distance_traveled) #ABS_Acel en cada km recorrido
    
    
    #Indice de dinamismo 
    i = (ABS_Acel / n_ia_t)*(total_time*(t_acel/100))
    
    
    # Crear el DataFrame con los resultados
    results_df = pd.DataFrame(columns=["Variable", "Valor"])
    results_df["Variable"] = ["distance_traveled", "total_time", "speed_mean", "speed_max", "speed_mean_sup",
                              "acel_pos_mean", "acel_pos_max", "n_ia_pos", "n_ia_neg", "n_ia_t", "RPA", "acel_neg_mean", "acel_neg_max",
                              "time_ralenti", "time_mov","t_crucero", "t_acel", "ABS_Der_Acel", "ABS_Der_Acel/distance", "ABS_Acel", 
                              "ABS_Acel/distance", "i"]
    results_df["Valor"] = [distance_traveled, total_time, speed_mean, speed_max, speed_mean_sup,
                            acel_pos_mean, acel_pos_max, n_datos_pos, n_datos_neg, n_ia_t, RPA, acel_neg_mean, acel_neg_max,
                            time_ralenti, time_mov,t_crucero,t_acel, ABS_Der_Acel, ABS_Der_Acel_Per_distance, ABS_Acel, 
                           ABS_Acel_Per_distance, i]
    # Redondear los valores a dos decimales
    results_df["Valor"] = results_df["Valor"].round(5)
    
    return results_df

In [4]:
### CALCULAR VARIABLES CICLOS

# Ruta del directorio donde se encuentran los archivos Excel originales
directorio = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles'
directorio_guardar = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f'

# Lista para almacenar los nombres de los archivos Excel
archivos_excel = []

# Lista para almacenar los DataFrames con los resultados
resultados = []

# Recorrer el directorio y guardar los nombres de los archivos Excel
for archivo in os.listdir(directorio):
    if archivo.endswith('.xlsx'):
        archivos_excel.append(archivo)

# Iterar sobre cada archivo Excel
for archivo in archivos_excel:
    # Ruta completa del archivo
    ruta_archivo = os.path.join(directorio, archivo)
    
    # Cargar el archivo Excel en un DataFrame
    df = pd.read_excel(ruta_archivo)
    df=df[df["intervalo"]=="Motorway"]
    # Aplicar la función para calcular los parámetros de aceleración
    df_modificado = calcular_parametros_acel(df)
 
    # Generar la ruta de destino para guardar el archivo modificado
    ruta_destino = os.path.join(directorio_guardar, archivo )
    
    # Guardar el DataFrame modificado como un nuevo archivo Excel
    df_modificado.to_excel(ruta_destino, index=False)

In [5]:
### CALCULAR PARAMETROS CICLOS

# Ruta del directorio donde se encuentran los archivos Excel originales
directorio = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f'
directorio_guardar = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_P'

# Lista para almacenar los nombres de los archivos Excel
archivos_excel = []

# Lista para almacenar los DataFrames con los resultados
resultados = []

# Recorrer el directorio y guardar los nombres de los archivos Excel
for archivo in os.listdir(directorio):
    if archivo.endswith('.xlsx'):
        archivos_excel.append(archivo)

# Iterar sobre cada archivo Excel
for archivo in archivos_excel:
    # Ruta completa del archivo
    ruta_archivo = os.path.join(directorio, archivo)
    
    # Cargar el archivo Excel en un DataFrame
    df = pd.read_excel(ruta_archivo)
    


    # Aplicar la función para calcular los parámetros de sección
    print(ruta_archivo)
    df_resultados = parametros_seccion(df)

    
    # Generar la ruta de destino para guardar el archivo modificado
    ruta_destino = os.path.join(directorio_guardar, "P_"+archivo )
    
    # Guardar el DataFrame modificado como un nuevo archivo Excel
    df_resultados.to_excel(ruta_destino, index=False)

C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f\EPA_HWFET.xlsx
C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f\EPA_US06.xlsx
C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f\NEDC ECE.xlsx
C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f\RDE_95.xlsx
C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_f\WLTC Class3.xlsx


In [6]:
def obtener_nombres_archivos(directorio, extension):
    return [archivo for archivo in os.listdir(directorio) if archivo.endswith(extension)]

def obtener_nombres_completos(nombres_archivos):
    return [os.path.splitext(archivo)[0] for archivo in nombres_archivos]

def obtener_nombres_finales(nombres_archivos):
    return [nombre]

# Directorio donde se encuentran los archivos Excel
directorio = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Other_cycles/Other_cycles_p'

# Obtener los nombres de los archivos Excel
archivos_excel = obtener_nombres_archivos(directorio, '.xlsx')

# Obtener los nombres base de los archivos (sin extensión)
nombres_archivos_base = obtener_nombres_completos(archivos_excel)

# # Obtener los nombres finales de los archivos
# Nombres_r = obtener_nombres_finales(nombres_archivos_base)

# Lista para almacenar los DataFrames
dataframes = []

# Cargar los archivos Excel y cambiar el nombre de la segunda columna
for nombre_archivo in archivos_excel:
    # Cargar el archivo Excel
    ruta_archivo = os.path.join(directorio, nombre_archivo)
    df = pd.read_excel(ruta_archivo)
    # Cambiar el nombre de la segunda columna
    df.columns = [df.columns[0], nombres_archivos_base.pop(0)]
    # Agregar el DataFrame a la lista
    dataframes.append(df)

# Combinar los DataFrames en uno solo
df_final = pd.concat([dataframes[0].iloc[:, :2]] + [df.iloc[:, 1] for df in dataframes[1:]], axis=1)
# Cambiar el nombre de la columna 'A' a 'NuevoNombre'
df_final = df_final.rename(columns={'Variable': 'Parameter'})
# Mostrar el DataFrame final

# print(df_final)

# guardar datos en un archivo :
df_final.to_excel(directorio + "/PorIntervalos/Par_Other_CiclosMotorway.xlsx", index=False)

In [7]:
# Transponer el DataFrame
df_final_T = df_final.transpose()
# Establecer la primera fila como nombres de columna
df_final_T.columns = df_final_T.iloc[0]
# Eliminar la primera fila (que ahora es el nombre de la columna)
df_final_T = df_final_T[1:]
# Actualizar Index y cambiar nombre a Ciclo
df_final_T.reset_index(inplace=True)
df_final_T.rename(columns={'index': 'ciclo'}, inplace=True)
# ver dataFrame
df_final_T
# guardar datos en un archivo :
df_final_T.to_excel(directorio + "/PorIntervalos/Par_Other_CiclosMotorwayT.xlsx", index=False)

In [12]:
import plotly.graph_objects as go
import numpy as np


# # Definir un diccionario que asigna formas a cada categoría
# shapes = {'Motorway_1': 'circle', 'Motorway_2': 'circle', 'Motorway_3': 'circle',
#           'Rural_1': 'x', 'Rural_2': 'x', 'Rural_3': 'x',
#           'Urban_1': 'square', 'Urban_2': 'square', 'Urban_3': 'square'}

# # Definir un diccionario que asigna colores a cada categoría
# colors = {'Motorway_1': 'darkblue', 'Motorway_2': 'darkorange', 'Motorway_3': 'darkred',
#           'Rural_1': 'darkred', 'Rural_2': 'darkorange', 'Rural_3': 'darkblue',
#           'Urban_1': 'darkorange', 'Urban_2': 'darkred', 'Urban_3': 'darkblue'}

# Crear el gráfico de dispersión con diferentes formas y colores para cada categoría
fig = go.Figure()

# Iterar sobre cada categoría en tu lista predefinida de categorías
for categoria in df_final_T['ciclo'].unique():
    # Filtrar el DataFrame por la categoría actual
    df_categoria = df_final_T[df_final_T['ciclo'] == categoria]
    
#     # Agregar una traza para la categoría actual con la forma y color correspondientes
#     fig.add_trace(go.Scatter(
#         x=df_categoria['RPA'],
#         y=df_categoria['i'],
#         mode='markers',
#         name=categoria,
#         marker=dict(symbol=shape, size=10, color=colors[categoria])
#     ))

    # Agregar una traza para la categoría actual con la forma y color correspondientes
    fig.add_trace(go.Scatter(
        x=df_categoria['RPA'],
        y=df_categoria['i'],
        mode='markers',
        name=categoria,
        marker=dict(symbol="x", size=10, color="black")
    ))
    
# Actualizar ejes
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', title_text="--") #"RPA [m/s²]"
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', title_text="IDI [m/s]")    
fig.update_layout(height=400, width=500, plot_bgcolor="white")
# Mostrar el gráfico
fig.show()


Separación de intervalos Urban, Rural, y Motorway de cada uno de los ciclos.  

In [9]:
df_final_T

Parameter,ciclo,distance_traveled,total_time,speed_mean,speed_max,speed_mean_sup,acel_pos_mean,acel_pos_max,n_ia_pos,n_ia_neg,...,acel_neg_max,time_ralenti,time_mov,t_crucero,t_acel,ABS_Der_Acel,ABS_Der_Acel/distance,ABS_Acel,ABS_Acel/distance,i
0,P_EPA_HWFET,16513.47185,765.0,77.60901,96.439,87.024,0.17179,1.44602,208.0,156.0,...,-1.53546,0.78431,99.34641,52.02614,47.32026,46.70491,0.00283,131.88956,0.00799,131.16489
1,P_EPA_US06,11673.61806,451.0,92.97572,129.283,111.12936,0.40009,3.5256,163.0,141.0,...,-2.84359,3.54767,96.67406,30.37694,66.29712,94.93037,0.00813,179.31375,0.01536,176.36451
2,P_NEDC ECE,6954.72234,399.0,62.5925,120.0,91.29625,0.19196,0.90278,111.0,45.0,...,-1.50463,10.27569,89.97494,51.62907,38.34586,14.46762,0.00208,79.47049,0.01143,77.94221
3,P_RDE_95,57042.59815,2800.0,73.3143,159.29,116.30215,0.52653,2.19259,1151.0,1057.0,...,-2.19051,2.0,98.03571,19.17857,78.85714,552.59028,0.00969,1568.13356,0.02749,1568.13356
4,P_WLTC Class3,15412.11111,777.0,71.31568,131.3,101.30784,0.3538,1.66204,263.0,251.0,...,-1.57639,4.7619,95.3668,29.72973,65.63707,80.4537,0.00522,261.81019,0.01699,259.77275
